In [5]:
import pandas as pd 
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam

# Load your data
df = pd.read_csv('antipyretic.csv')

# Convert 'Sold_date' to datetime format
df['Sold_date'] = pd.to_datetime(df['Sold_date'], format='%m/%d/%y')

# Set 'Sold_date' as the index
df.set_index('Sold_date', inplace=True)

# Aggregate sales data on a weekly basis for each product
weekly_sales = df.groupby([pd.Grouper(freq='W'), 
'Product_details']).agg(total_sold=('Sold_quantity', 'sum')).reset_index()

# Apply one-hot encoding to 'Product_details'
weekly_sales_encoded = pd.concat([weekly_sales, pd.get_dummies(weekly_sales['Product_details'], 
prefix='product')], axis=1).drop('Product_details', axis=1)

# Extract year and week number from 'Sold_date' for temporal features
weekly_sales_encoded['year'] = weekly_sales_encoded['Sold_date'].dt.year
weekly_sales_encoded['week_of_year'] = weekly_sales_encoded['Sold_date'].dt.isocalendar().week

# Normalize the temporal features 
scaler = MinMaxScaler()
features_columns = ['year', 'week_of_year'] + [col for col in weekly_sales_encoded.columns 
if col.startswith('product')]
weekly_sales_encoded[features_columns] = scaler.fit_transform(weekly_sales_encoded[features_columns])

# Split the data into features and target
X = weekly_sales_encoded[features_columns].values
y = weekly_sales_encoded['total_sold'].values

# Adjusting the dataset split into 70% training, 15% validation, and 15% testing
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.15, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.1765, random_state=42)  # 0.1765 of 85% is roughly 15% of the whole

# Define the model architecture
model = Sequential([
    Input(shape=(X_train.shape[1],)),
    Dense(100, activation='relu'),  # Additional hidden layer
    Dense(100, activation='relu'),  # Additional hidden layer
    Dense(100, activation='relu'),  # Additional hidden layer
    Dense(1, activation='relu')    # Output layer
])


# Compile the model
model.compile(optimizer=Adam(), loss='mean_squared_error')

# Train the model with the validation data
model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=100, batch_size=10)

# Evaluate the model
predictions = model.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, predictions))
print(f'RMSE on test set: {rmse}')

# Prepare forecasting features for Feb 1 to 8, 2024, correctly this time
forecast_results = []
for product_col in [col for col in features_columns if col.startswith('product')]:
    forecast_period_features = np.zeros((1, len(features_columns)))
    # Find the index of the year, week_of_year, and current product column
    year_index = features_columns.index('year')
    week_of_year_index = features_columns.index('week_of_year')
    product_index = features_columns.index(product_col)
    
    # Normalize year and week_of_year values
    forecast_period_features[0, year_index] = (2024 - scaler.data_min_[year_index]) / (scaler.data_max_[year_index] - scaler.data_min_[year_index])
    forecast_period_features[0, week_of_year_index] = (5 - scaler.data_min_[week_of_year_index]) / (scaler.data_max_[week_of_year_index] - scaler.data_min_[week_of_year_index])
    forecast_period_features[0, product_index] = 1  # Activate current product

    # Predict and store the result
    predicted_quantity = model.predict(forecast_period_features).flatten()[0]
    product_name = product_col.replace('product_', '')  # Assuming prefix 'product_' is used
    forecast_results.append((product_name, predicted_quantity))

# Display forecasted quantities for each product in a DataFrame
forecast_df = pd.DataFrame(forecast_results, columns=['Product', 'Forecasted Quantity']).sort_values(by='Forecasted Quantity', ascending=False)
print(forecast_df)

Epoch 1/100
188/188 ━━━━━━━━━━━━━━━━━━━━ 0s 560us/step - loss: 6667.7085 - val_loss: 3633.0618
Epoch 2/100
188/188 ━━━━━━━━━━━━━━━━━━━━ 0s 363us/step - loss: 6249.3291 - val_loss: 3315.6890
Epoch 3/100
188/188 ━━━━━━━━━━━━━━━━━━━━ 0s 358us/step - loss: 5929.9297 - val_loss: 3558.1606
Epoch 4/100
188/188 ━━━━━━━━━━━━━━━━━━━━ 0s 396us/step - loss: 2552.8901 - val_loss: 3304.6997
Epoch 5/100
188/188 ━━━━━━━━━━━━━━━━━━━━ 0s 409us/step - loss: 6232.7056 - val_loss: 3348.2410
Epoch 6/100
188/188 ━━━━━━━━━━━━━━━━━━━━ 0s 442us/step - loss: 4882.3608 - val_loss: 3744.9634
Epoch 7/100
188/188 ━━━━━━━━━━━━━━━━━━━━ 0s 407us/step - loss: 5339.7241 - val_loss: 4145.9321
Epoch 8/100
188/188 ━━━━━━━━━━━━━━━━━━━━ 0s 418us/step - loss: 7335.6216 - val_loss: 3452.7214
Epoch 9/100
188/188 ━━━━━━━━━━━━━━━━━━━━ 0s 365us/step - loss: 3279.4548 - val_loss: 3437.7930
Epoch 10/100
188/188 ━━━━━━━━━━━━━━━━━━━━ 0s 354us/step - loss: 3812.2607 - val_loss: 3461.2185
Epoch 11/100
188/188 ━━━━━━━━━━━━━━━━━━━━ 0s 447u

188/188 ━━━━━━━━━━━━━━━━━━━━ 0s 356us/step - loss: 745.6178 - val_loss: 2684.7952
Epoch 65/100
188/188 ━━━━━━━━━━━━━━━━━━━━ 0s 373us/step - loss: 891.4213 - val_loss: 2474.1875
Epoch 66/100
188/188 ━━━━━━━━━━━━━━━━━━━━ 0s 359us/step - loss: 1591.4606 - val_loss: 2961.8740
Epoch 67/100
188/188 ━━━━━━━━━━━━━━━━━━━━ 0s 362us/step - loss: 3144.8584 - val_loss: 2943.5530
Epoch 68/100
188/188 ━━━━━━━━━━━━━━━━━━━━ 0s 358us/step - loss: 1606.0593 - val_loss: 2990.7407
Epoch 69/100
188/188 ━━━━━━━━━━━━━━━━━━━━ 0s 354us/step - loss: 1186.9824 - val_loss: 2994.8330
Epoch 70/100
188/188 ━━━━━━━━━━━━━━━━━━━━ 0s 353us/step - loss: 839.8896 - val_loss: 2779.2300
Epoch 71/100
188/188 ━━━━━━━━━━━━━━━━━━━━ 0s 354us/step - loss: 1762.0037 - val_loss: 2728.7720
Epoch 72/100
188/188 ━━━━━━━━━━━━━━━━━━━━ 0s 354us/step - loss: 1401.7994 - val_loss: 2576.9922
Epoch 73/100
188/188 ━━━━━━━━━━━━━━━━━━━━ 0s 354us/step - loss: 1301.4373 - val_loss: 2793.3860
Epoch 74/100
188/188 ━━━━━━━━━━━━━━━━━━━━ 0s 365us/step 